In [0]:
# Install dependency, neuprint-python
!pip install neuprint-python

In [0]:
import matplotlib.pyplot as plt
import neuprint
import numpy as np
import pandas as pd

In [0]:
# Connect to google drive, google sheets
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
# Load workbook from google sheets
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1jqbNL18jvMspH0GO_7q-X7Euqsi_1eAtLoac2PZc6PY/edit#gid=0')
ln_sheet = wb.worksheet('LNs')
LNs = ln_sheet.get_all_values()
LNs = pd.DataFrame(LNs)
LNs.columns = LNs.iloc[0]
LNs = LNs.iloc[1:]

In [0]:
# Establish connection to neuprint
